In [256]:
cd ../task_01/pipelines

[Errno 2] No such file or directory: '../task_01/pipelines'
/Users/pldelacour/Documents/PL_Ecole/data_sc_lab/data_sc_lab/task_01/pipelines


In [257]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ipynb.fs.full.tool_functions import *
from yellowbrick.regressor import AlphaSelection
from sklearn.model_selection import train_test_split
from numpy import mean
from numpy import std
from sklearn.model_selection import RepeatedKFold
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler, ReduceLROnPlateau

In [317]:
df_beha = pd.read_csv('../../../data/Behavioral/AllData.csv')#, nrows=1000)

df_spectro_cluster = pd.read_csv("../../../data/EEG/RestingEEG_Spectro_Cluster.csv") #, nrows=1000)

srs = pd.read_csv('../../../data/HBN_srs.csv', delimiter=';')
outliers = pd.read_csv('../../../data/outliers.csv')

/Users/pldelacour/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (66,68,80,81,82,83,84,95,96,97,98,99,110,111,112,113,114,125,126,127,128,129,140,141,142,144,155,156,157,159,932,933,934,935,936,937,938,939,940,941,942,943,944,945,946,947,948,949,950,951,952,953,954,955,956,957,958,959,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999,1000,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,1011,1012,1013,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023,1024,1025,1026,1027,1028,1029,1030,1031,1032,1033,1034,1035,1036,1037,1038,1039,1040,1041,1042,1043,1044,1045,1046,1047,1048,1049,1050,1051,1052,1053,1054,1055,1056,1057,1058,1059,1060,1061,1062,1063,1064,1065,1066,1067,1068,1069,1070,1071,1072,1073,1074,1075,1076,1077,1078,1079,1080,1081,1082,1083,1084,1085,1086,1087,1088,1089,1090,1091,1092,1093,1094,1095,1096,1097,1098,1099,11

In [259]:
#s_in = df_beha[['SWAN_IN_Avg' , 'Age', 'Patient_ID']]
#s_HY = df_beha[['SWAN_HY_Avg' , 'Age', 'Patient_ID']]

In [260]:
#df_beha[['Sex']]

## SWAN

In [325]:
swan = df_beha[['SWAN_IN_Avg' , 'SWAN_HY_Avg', 'Patient_ID']]
in_null = swan['SWAN_IN_Avg'].isnull().sum()
hy_null = swan['SWAN_HY_Avg'].isnull().sum()
#print(in_null)
#print(hy_null)
swan = swan[~ (swan['SWAN_IN_Avg'].isnull() | swan['SWAN_HY_Avg'].isnull()) ]

## SRS

In [329]:
srs = srs[~srs['SRS_RRB'].isnull()]
srs.rename(columns={'IDs': 'Patient_ID'}, inplace=True)

In [330]:
scores = pd.merge(swan , srs , on = 'Patient_ID')
scores

,SWAN_IN_Avg,SWAN_HY_Avg,Patient_ID,Sex,Age,SRS_RRB,SRS_SCI,SRS_Total
0,0.000000,1.222222,NDARYM832PX3,1,7.048254,10.0,45.0,55.0
1,1.222222,0.555555,NDARNJ687DMC,1,6.348163,19.0,73.0,92.0
2,1.111111,1.888888,NDARED632KNG,1,17.203855,2.0,28.0,30.0
3,-0.111111,2.222222,NDARFM080VAF,1,7.623203,14.0,72.0,86.0
4,-0.111111,0.777777,NDARFW130NGG,1,8.316107,25.0,85.0,110.0
...,...,...,...,...,...,...,...,...
1234,-1.444444,-2.333333,NDARND697FLK,1,6.517796,1.0,17.0,18.0
1235,1.222222,0.444444,NDARCL008HLA,0,16.456194,21.0,82.0,103.0
1236,-1.888888,-3.000000,NDARYM334BZ5,0,16.119438,0.0,6.0,6.0
1237,-0.666666,1.333333,NDARYZ986HEW,1,7.563084,9.0,49.0,58.0


## Combine with spectro cluster 

In [331]:
df_spectro_cluster.rename(columns={'id': 'Patient_ID'}, inplace=True)
df_label_spectro_cluster = pd.merge(scores,  df_spectro_cluster, on='Patient_ID')

df_label_spectro_cluster = fill_with_median(df_label_spectro_cluster)

In [332]:
less_18 = df_label_spectro_cluster['Age'] <= 18.0
df_label_spectro_cluster = df_label_spectro_cluster[less_18]

In [333]:
df_label_spectro_cluster

,SWAN_IN_Avg,SWAN_HY_Avg,Patient_ID,Sex,Age,SRS_RRB,SRS_SCI,SRS_Total,eyesclosed_fband_delta_absmean_lfront,eyesclosed_fband_delta_absmean_mfront,...,eyesopen_fooof_peak_freq_rfront,eyesopen_fooof_peak_freq_lpari,eyesopen_fooof_peak_freq_mpari,eyesopen_fooof_peak_freq_rpari,eyesopen_fooof_peak_amplitude_lfront,eyesopen_fooof_peak_amplitude_mfront,eyesopen_fooof_peak_amplitude_rfront,eyesopen_fooof_peak_amplitude_lpari,eyesopen_fooof_peak_amplitude_mpari,eyesopen_fooof_peak_amplitude_rpari
0,-0.111111,2.222222,NDARFM080VAF,1,7.623203,14.0,72.0,86.0,7.519803,7.043891,...,9.485781,9.441022,9.384172,9.425798,0.795549,0.876262,0.872379,0.989101,1.099981,0.915441
1,-0.444444,0.444444,NDARVG148WHV,0,10.835158,17.0,70.0,87.0,2.346685,3.945786,...,8.800915,8.510134,9.078390,14.741810,0.254701,0.351010,0.303875,0.234960,0.232564,0.344537
2,1.333333,1.444444,NDARUY876VL1,0,7.987451,19.0,57.0,76.0,1.281869,1.229748,...,5.604535,5.602136,4.096616,5.084040,0.048655,0.233294,0.282735,0.256238,0.175123,0.304286
3,0.666666,-0.888888,NDARYU323ZDJ,1,8.096851,12.0,58.0,70.0,3.109075,4.803264,...,9.288451,6.651558,6.396867,10.291086,0.218827,0.313734,0.298905,0.178899,0.108880,0.449437
4,0.111111,1.222222,NDARCP360AFD,0,8.357061,15.0,68.0,83.0,5.786769,6.818487,...,6.952035,5.065536,4.679294,4.924409,0.150518,0.169058,0.154417,0.104574,0.102992,0.094126
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
901,-1.444444,-2.333333,NDARND697FLK,1,6.517796,1.0,17.0,18.0,8.594610,13.258798,...,5.868008,22.846455,7.050818,4.622321,0.416442,0.358625,0.380528,0.169463,0.325197,0.199935
902,1.222222,0.444444,NDARCL008HLA,0,16.456194,21.0,82.0,103.0,7.487871,9.809903,...,9.389994,9.540163,9.482167,9.545163,1.326583,1.288238,1.407225,1.561878,1.636154,1.412149
903,-1.888888,-3.000000,NDARYM334BZ5,0,16.119438,0.0,6.0,6.0,3.743134,3.795633,...,10.376962,10.707490,10.383040,10.343315,0.623514,0.582917,0.422394,0.615360,0.860283,0.945789
904,-0.666666,1.333333,NDARYZ986HEW,1,7.563084,9.0,49.0,58.0,2.830657,5.259391,...,9.482208,7.101976,9.409898,9.301044,0.185029,0.165101,0.246363,0.120704,0.254212,0.271848


In [334]:
# Remove preidentified outliers

In [335]:
outliers = pd.read_csv('../../../data/outliers.csv')
df_label_spectro_cluster = df_label_spectro_cluster[~df_label_spectro_cluster.Patient_ID.isin(outliers.Patient_ID)]

## Train Test split 

In [336]:
X = df_label_spectro_cluster.drop([ 'SWAN_IN_Avg' , 'SWAN_HY_Avg', 'SRS_Total' , 'SRS_RRB' , 'SRS_SCI', 'Patient_ID'], axis=1)
Y = df_label_spectro_cluster[[ 'SWAN_IN_Avg' , 'SWAN_HY_Avg', 'SRS_Total' , 'SRS_RRB' , 'SRS_SCI']]


In [337]:
X

,Sex,Age,eyesclosed_fband_delta_absmean_lfront,eyesclosed_fband_delta_absmean_mfront,eyesclosed_fband_delta_absmean_rfront,eyesclosed_fband_delta_absmean_lpari,eyesclosed_fband_delta_absmean_mpari,eyesclosed_fband_delta_absmean_rpari,eyesclosed_fband_delta_relmean_lfront,eyesclosed_fband_delta_relmean_mfront,...,eyesopen_fooof_peak_freq_rfront,eyesopen_fooof_peak_freq_lpari,eyesopen_fooof_peak_freq_mpari,eyesopen_fooof_peak_freq_rpari,eyesopen_fooof_peak_amplitude_lfront,eyesopen_fooof_peak_amplitude_mfront,eyesopen_fooof_peak_amplitude_rfront,eyesopen_fooof_peak_amplitude_lpari,eyesopen_fooof_peak_amplitude_mpari,eyesopen_fooof_peak_amplitude_rpari
0,1,7.623203,7.519803,7.043891,6.528158,9.633529,9.989483,9.305804,9.598653,10.164125,...,9.485781,9.441022,9.384172,9.425798,0.795549,0.876262,0.872379,0.989101,1.099981,0.915441
1,0,10.835158,2.346685,3.945786,2.962716,3.440606,3.565384,3.023286,10.968235,12.461058,...,8.800915,8.510134,9.078390,14.741810,0.254701,0.351010,0.303875,0.234960,0.232564,0.344537
2,0,7.987451,1.281869,1.229748,1.905450,1.768091,1.484286,0.756394,13.132905,11.998897,...,5.604535,5.602136,4.096616,5.084040,0.048655,0.233294,0.282735,0.256238,0.175123,0.304286
3,1,8.096851,3.109075,4.803264,2.873502,2.644382,3.884837,2.391569,9.931212,10.869944,...,9.288451,6.651558,6.396867,10.291086,0.218827,0.313734,0.298905,0.178899,0.108880,0.449437
4,0,8.357061,5.786769,6.818487,6.296279,8.043145,9.413683,6.839451,10.495082,11.095547,...,6.952035,5.065536,4.679294,4.924409,0.150518,0.169058,0.154417,0.104574,0.102992,0.094126
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
901,1,6.517796,8.594610,13.258798,7.158682,12.743212,8.698643,4.855232,15.315470,14.779320,...,5.868008,22.846455,7.050818,4.622321,0.416442,0.358625,0.380528,0.169463,0.325197,0.199935
902,0,16.456194,7.487871,9.809903,7.255467,8.731814,9.629680,11.326718,7.404931,7.060421,...,9.389994,9.540163,9.482167,9.545163,1.326583,1.288238,1.407225,1.561878,1.636154,1.412149
903,0,16.119438,3.743134,3.795633,3.286381,2.779438,3.539956,3.800533,8.565498,8.383859,...,10.376962,10.707490,10.383040,10.343315,0.623514,0.582917,0.422394,0.615360,0.860283,0.945789
904,1,7.563084,2.830657,5.259391,6.724664,4.745513,5.885628,3.160138,13.388712,13.121943,...,9.482208,7.101976,9.409898,9.301044,0.185029,0.165101,0.246363,0.120704,0.254212,0.271848


In [338]:

x_train , x_test, y_train , y_test = train_test_split(X,Y, test_size=0.2, random_state = 0)


In [339]:
forest = IsolationForest()
forest.fit(x_train)

# Outlier indices for training
outliers_training = forest.predict(x_train)
outliers_training_indices = np.argwhere(outliers_training == 1).flatten()

# Drop signal outliers in training data
x_train = x_train[outliers_training == 1]
y_train = y_train[outliers_training == 1]
#x_train, y_train = drop_outliers_samples_isolation_forest(x_train, y_train)
print(outliers_training_indices)
print(x_test.shape)
print(y_test.shape)
print(x_train.shape)
print(y_train.shape)

[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  17  18
  19  20  22  23  25  26  27  28  29  30  31  32  33  34  35  36  37  38
  39  40  41  42  43  44  45  47  49  50  51  52  53  54  55  57  58  59
  60  61  62  63  64  65  66  67  68  69  70  71  72  73  74  75  76  77
  78  79  80  81  82  83  84  85  86  87  88  89  90  91  92  93  94  95
  97  98  99 100 101 102 103 104 105 106 107 108 109 110 111 113 114 115
 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133
 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151
 152 153 154 155 156 157 158 159 161 162 163 164 165 166 167 168 169 170
 171 172 173 174 175 176 177 178 179 181 182 183 184 185 186 187 188 189
 190 191 192 193 194 195 196 197 198 199 200 201 203 204 205 207 208 209
 210 211 212 213 214 215 216 218 219 220 221 222 223 224 225 226 227 229
 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247
 248 249 250 251 252 253 254 255 256 257 258 259 26

In [340]:
scaler = StandardScaler()
scaler.fit(x_train)
x_train = pd.DataFrame(scaler.transform(x_train), columns=x_train.columns, index=x_train.index)
x_test = pd.DataFrame(scaler.transform(x_test), columns=x_test.columns, index=x_test.index)


## Model

# Scaling the y

In [341]:
def normalize(y) : 
    y_mean = np.mean(y , axis = 0)
    y_std = np.std(y , axis = 0)
    y_scaled = (y - y_mean)/y_std
    return (y_scaled , y_mean , y_std )
def normalize_p(y , m , s):
    return (y - m)/s
def denormalize( y_scaled ,y_mean , y_std ):
    y = y_scaled*y_std +y_mean
    return y

In [342]:
y_train_scaled , y_mean, y_std ,  = normalize(y_train)


In [343]:
# get the dataset
 
# get the model
def get_model(n_inputs, n_outputs):
    model = Sequential()
    model.add(Dense(50, input_dim=n_inputs, kernel_initializer='he_uniform', activation='relu'))
    model.add(Dense(n_outputs))
    model.compile(loss='mse' , optimizer='adam' , metrics = loss_multi)
    return model
 
# evaluate a model using repeated k-fold cross-validation
def evaluate_model(X, y):
    results = list()
    n_inputs, n_outputs = X.shape[1], y.shape[1]
    
    # define evaluation procedure
    cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
    # enumerate folds
    for train_ix, test_ix in cv.split(X):
        # prepare data
        X_train, X_test = X[train_ix], X[test_ix]
        y_train, y_test = y[train_ix], y[test_ix]
        # define model
        model = get_model(n_inputs, n_outputs)
        # fit model
        model.fit(X_train, y_train, verbose=0, epochs=100)
        # evaluate model on test set
        mae = model.evaluate(X_test, y_test, verbose=0)
        # store result
        print('>%.3f' % mae)
    results.append(mae)
    return results


In [344]:
#results = evaluate_model(x_train.to_numpy(), y_train.to_numpy())
# summarize performance
#print('MAE: %.3f (%.3f)' % (mean(results), std(results)))

In [345]:
checkpointer = ModelCheckpoint(filepath='Best_model.h5', monitor='val_mse', verbose=1, save_best_only=True)
model = get_model(x_train.shape[1] , y_train.shape[1])
file_path = "best_model.h5"
checkpoint = ModelCheckpoint(file_path, monitor='mse', verbose=1, save_best_only=True, mode='max')

###
early = EarlyStopping(monitor="mse", mode="max", patience=10, verbose=1)
redonplat = ReduceLROnPlateau(monitor="mse", mode="max", patience=3, verbose=2)
callbacks_list = [checkpoint, early, redonplat]  # early
###
hist = model.fit(
    x_train.to_numpy(), 
    y_train_scaled.to_numpy(), 
    validation_split = 1/6,
    batch_size=275, epochs=400, verbose=2, shuffle=True, 
    callbacks=[checkpointer])

Train on 560 samples, validate on 113 samples
Epoch 1/400
 - 0s - loss: 2.4061 - mse: 2.4061 - val_loss: 1.7578 - val_mse: 1.7578

Epoch 00001: val_mse improved from inf to 1.75783, saving model to Best_model.h5
Epoch 2/400
 - 0s - loss: 1.7092 - mse: 1.7092 - val_loss: 1.4941 - val_mse: 1.4941

Epoch 00002: val_mse improved from 1.75783 to 1.49415, saving model to Best_model.h5
Epoch 3/400
 - 0s - loss: 1.5233 - mse: 1.5233 - val_loss: 1.4005 - val_mse: 1.4005

Epoch 00003: val_mse improved from 1.49415 to 1.40048, saving model to Best_model.h5
Epoch 4/400
 - 0s - loss: 1.4145 - mse: 1.4145 - val_loss: 1.3318 - val_mse: 1.3318

Epoch 00004: val_mse improved from 1.40048 to 1.33183, saving model to Best_model.h5
Epoch 5/400
 - 0s - loss: 1.3308 - mse: 1.3308 - val_loss: 1.3312 - val_mse: 1.3312

Epoch 00005: val_mse improved from 1.33183 to 1.33123, saving model to Best_model.h5
Epoch 6/400
 - 0s - loss: 1.2930 - mse: 1.2930 - val_loss: 1.3791 - val_mse: 1.3791

Epoch 00006: val_mse di


Epoch 00056: val_mse did not improve from 1.03596
Epoch 57/400
 - 0s - loss: 0.7948 - mse: 0.7948 - val_loss: 1.0837 - val_mse: 1.0837

Epoch 00057: val_mse did not improve from 1.03596
Epoch 58/400
 - 0s - loss: 0.7799 - mse: 0.7799 - val_loss: 1.0860 - val_mse: 1.0860

Epoch 00058: val_mse did not improve from 1.03596
Epoch 59/400
 - 0s - loss: 0.7758 - mse: 0.7758 - val_loss: 1.0810 - val_mse: 1.0810

Epoch 00059: val_mse did not improve from 1.03596
Epoch 60/400
 - 0s - loss: 0.7720 - mse: 0.7720 - val_loss: 1.0673 - val_mse: 1.0673

Epoch 00060: val_mse did not improve from 1.03596
Epoch 61/400
 - 0s - loss: 0.7642 - mse: 0.7642 - val_loss: 1.0527 - val_mse: 1.0527

Epoch 00061: val_mse did not improve from 1.03596
Epoch 62/400
 - 0s - loss: 0.7640 - mse: 0.7640 - val_loss: 1.0555 - val_mse: 1.0555

Epoch 00062: val_mse did not improve from 1.03596
Epoch 63/400
 - 0s - loss: 0.7555 - mse: 0.7555 - val_loss: 1.1105 - val_mse: 1.1105

Epoch 00063: val_mse did not improve from 1.035

 - 0s - loss: 0.6016 - mse: 0.6016 - val_loss: 1.1072 - val_mse: 1.1072

Epoch 00117: val_mse did not improve from 1.03596
Epoch 118/400
 - 0s - loss: 0.5928 - mse: 0.5928 - val_loss: 1.1042 - val_mse: 1.1042

Epoch 00118: val_mse did not improve from 1.03596
Epoch 119/400
 - 0s - loss: 0.5822 - mse: 0.5822 - val_loss: 1.0942 - val_mse: 1.0942

Epoch 00119: val_mse did not improve from 1.03596
Epoch 120/400
 - 0s - loss: 0.5825 - mse: 0.5825 - val_loss: 1.0906 - val_mse: 1.0906

Epoch 00120: val_mse did not improve from 1.03596
Epoch 121/400
 - 0s - loss: 0.5795 - mse: 0.5795 - val_loss: 1.1031 - val_mse: 1.1031

Epoch 00121: val_mse did not improve from 1.03596
Epoch 122/400
 - 0s - loss: 0.5746 - mse: 0.5746 - val_loss: 1.1301 - val_mse: 1.1301

Epoch 00122: val_mse did not improve from 1.03596
Epoch 123/400
 - 0s - loss: 0.5828 - mse: 0.5828 - val_loss: 1.1283 - val_mse: 1.1283

Epoch 00123: val_mse did not improve from 1.03596
Epoch 124/400
 - 0s - loss: 0.5864 - mse: 0.5864 - val_

 - 0s - loss: 0.4592 - mse: 0.4592 - val_loss: 1.1925 - val_mse: 1.1925

Epoch 00177: val_mse did not improve from 1.03596
Epoch 178/400
 - 0s - loss: 0.4493 - mse: 0.4493 - val_loss: 1.1569 - val_mse: 1.1569

Epoch 00178: val_mse did not improve from 1.03596
Epoch 179/400
 - 0s - loss: 0.4447 - mse: 0.4447 - val_loss: 1.1487 - val_mse: 1.1487

Epoch 00179: val_mse did not improve from 1.03596
Epoch 180/400
 - 0s - loss: 0.4479 - mse: 0.4479 - val_loss: 1.1555 - val_mse: 1.1555

Epoch 00180: val_mse did not improve from 1.03596
Epoch 181/400
 - 0s - loss: 0.4566 - mse: 0.4566 - val_loss: 1.1918 - val_mse: 1.1918

Epoch 00181: val_mse did not improve from 1.03596
Epoch 182/400
 - 0s - loss: 0.4542 - mse: 0.4542 - val_loss: 1.2259 - val_mse: 1.2259

Epoch 00182: val_mse did not improve from 1.03596
Epoch 183/400
 - 0s - loss: 0.4432 - mse: 0.4432 - val_loss: 1.2486 - val_mse: 1.2486

Epoch 00183: val_mse did not improve from 1.03596
Epoch 184/400
 - 0s - loss: 0.4516 - mse: 0.4516 - val_

Epoch 237/400
 - 0s - loss: 0.3668 - mse: 0.3668 - val_loss: 1.3119 - val_mse: 1.3119

Epoch 00237: val_mse did not improve from 1.03596
Epoch 238/400
 - 0s - loss: 0.3701 - mse: 0.3701 - val_loss: 1.2469 - val_mse: 1.2469

Epoch 00238: val_mse did not improve from 1.03596
Epoch 239/400
 - 0s - loss: 0.3523 - mse: 0.3523 - val_loss: 1.2414 - val_mse: 1.2414

Epoch 00239: val_mse did not improve from 1.03596
Epoch 240/400
 - 0s - loss: 0.3650 - mse: 0.3650 - val_loss: 1.2701 - val_mse: 1.2701

Epoch 00240: val_mse did not improve from 1.03596
Epoch 241/400
 - 0s - loss: 0.3654 - mse: 0.3654 - val_loss: 1.2891 - val_mse: 1.2891

Epoch 00241: val_mse did not improve from 1.03596
Epoch 242/400
 - 0s - loss: 0.3506 - mse: 0.3506 - val_loss: 1.2706 - val_mse: 1.2706

Epoch 00242: val_mse did not improve from 1.03596
Epoch 243/400
 - 0s - loss: 0.3510 - mse: 0.3510 - val_loss: 1.2715 - val_mse: 1.2715

Epoch 00243: val_mse did not improve from 1.03596
Epoch 244/400
 - 0s - loss: 0.3502 - mse:

 - 0s - loss: 0.2759 - mse: 0.2759 - val_loss: 1.3298 - val_mse: 1.3298

Epoch 00297: val_mse did not improve from 1.03596
Epoch 298/400
 - 0s - loss: 0.2564 - mse: 0.2564 - val_loss: 1.3375 - val_mse: 1.3375

Epoch 00298: val_mse did not improve from 1.03596
Epoch 299/400
 - 0s - loss: 0.2705 - mse: 0.2705 - val_loss: 1.3653 - val_mse: 1.3653

Epoch 00299: val_mse did not improve from 1.03596
Epoch 300/400
 - 0s - loss: 0.2621 - mse: 0.2621 - val_loss: 1.4044 - val_mse: 1.4044

Epoch 00300: val_mse did not improve from 1.03596
Epoch 301/400
 - 0s - loss: 0.2638 - mse: 0.2638 - val_loss: 1.3854 - val_mse: 1.3854

Epoch 00301: val_mse did not improve from 1.03596
Epoch 302/400
 - 0s - loss: 0.2566 - mse: 0.2566 - val_loss: 1.3402 - val_mse: 1.3402

Epoch 00302: val_mse did not improve from 1.03596
Epoch 303/400
 - 0s - loss: 0.2764 - mse: 0.2764 - val_loss: 1.3498 - val_mse: 1.3498

Epoch 00303: val_mse did not improve from 1.03596
Epoch 304/400
 - 0s - loss: 0.2562 - mse: 0.2562 - val_

 - 0s - loss: 0.2338 - mse: 0.2338 - val_loss: 1.4604 - val_mse: 1.4604

Epoch 00357: val_mse did not improve from 1.03596
Epoch 358/400
 - 0s - loss: 0.2225 - mse: 0.2225 - val_loss: 1.4368 - val_mse: 1.4368

Epoch 00358: val_mse did not improve from 1.03596
Epoch 359/400
 - 0s - loss: 0.2100 - mse: 0.2100 - val_loss: 1.4107 - val_mse: 1.4107

Epoch 00359: val_mse did not improve from 1.03596
Epoch 360/400
 - 0s - loss: 0.2065 - mse: 0.2065 - val_loss: 1.4182 - val_mse: 1.4182

Epoch 00360: val_mse did not improve from 1.03596
Epoch 361/400
 - 0s - loss: 0.2070 - mse: 0.2070 - val_loss: 1.4418 - val_mse: 1.4418

Epoch 00361: val_mse did not improve from 1.03596
Epoch 362/400
 - 0s - loss: 0.2147 - mse: 0.2147 - val_loss: 1.4104 - val_mse: 1.4104

Epoch 00362: val_mse did not improve from 1.03596
Epoch 363/400
 - 0s - loss: 0.2034 - mse: 0.2034 - val_loss: 1.4179 - val_mse: 1.4179

Epoch 00363: val_mse did not improve from 1.03596
Epoch 364/400
 - 0s - loss: 0.2001 - mse: 0.2001 - val_


$\sum\limits_{i = 1}^n \| y_i -  \hat{y_i}\|^2$

In [346]:
y_pred = model.predict(x_test.to_numpy())
y_test_n = normalize_p(y_test,y_mean , y_std)

## Mean square error 

In [347]:
from sklearn.metrics import mean_squared_error

In [348]:
#y_pred = denormalize(y_pred , np.array(y_mean) , np.array(y_std))
for i in range(len(y_test.columns)):
    print(y_test.columns[i] , ':', mean_squared_error( y_test_n.to_numpy()[:,i] , y_pred[:,i]) )

SWAN_IN_Avg : 1.9966317777124991
SWAN_HY_Avg : 1.7786173711882058
SRS_Total : 1.673988663716875
SRS_RRB : 1.4446659676323625
SRS_SCI : 1.597689067197557


# Mean Baseline

In [349]:
y_baseline = np.zeros(y_test_n.shape) + np.mean(y_train_scaled.to_numpy() , axis=0)

for i in range(len(y_test.columns)):
    print(y_test.columns[i] , ':', mean_squared_error( y_test_n.to_numpy()[:,i] , y_baseline[:,i]) )

SWAN_IN_Avg : 1.0000256644070629
SWAN_HY_Avg : 0.9728785044418361
SRS_Total : 0.9163086905500325
SRS_RRB : 0.8761106624325128
SRS_SCI : 0.9179364884401283


## Using the Latent Space 

In [191]:
pwd

'/Users/pldelacour/Documents/PL_Ecole/data_sc_lab/data_sc_lab/task_01/pipelines'

In [350]:
df_x_train = pd.read_csv("../../../data/x_train_latent.csv")
df_x_test = pd.read_csv("../../../data/x_test_latent.csv")
df_beha = pd.read_csv('../../../data/Behavioral/AllData.csv')
srs = pd.read_csv('../../../data/HBN_srs.csv', delimiter=';')
outliers = pd.read_csv('../../../data/outliers.csv')

/Users/pldelacour/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (66,68,80,81,82,83,84,95,96,97,98,99,110,111,112,113,114,125,126,127,128,129,140,141,142,144,155,156,157,159,932,933,934,935,936,937,938,939,940,941,942,943,944,945,946,947,948,949,950,951,952,953,954,955,956,957,958,959,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999,1000,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,1011,1012,1013,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023,1024,1025,1026,1027,1028,1029,1030,1031,1032,1033,1034,1035,1036,1037,1038,1039,1040,1041,1042,1043,1044,1045,1046,1047,1048,1049,1050,1051,1052,1053,1054,1055,1056,1057,1058,1059,1060,1061,1062,1063,1064,1065,1066,1067,1068,1069,1070,1071,1072,1073,1074,1075,1076,1077,1078,1079,1080,1081,1082,1083,1084,1085,1086,1087,1088,1089,1090,1091,1092,1093,1094,1095,1096,1097,1098,1099,11

In [351]:
df_x_train = df_x_train[~df_x_train.Patient_ID.isin(outliers.Patient_ID)]
df_x_test = df_x_test[~df_x_test.Patient_ID.isin(outliers.Patient_ID)]

## Swan

In [353]:
swan = df_beha[['SWAN_IN_Avg' , 'SWAN_HY_Avg',  'Patient_ID']]
in_null = swan['SWAN_IN_Avg'].isnull().sum()
hy_null = swan['SWAN_HY_Avg'].isnull().sum()
#print(in_null)
#print(hy_null)
swan = swan[~ (swan['SWAN_IN_Avg'].isnull() | swan['SWAN_HY_Avg'].isnull()) ]

## SRS

In [354]:
srs = srs[~srs['SRS_RRB'].isnull()]

In [355]:
srs.rename(columns={'IDs': 'Patient_ID'}, inplace=True)

In [356]:
scores = pd.merge(swan , srs , on = 'Patient_ID')
scores

,SWAN_IN_Avg,SWAN_HY_Avg,Patient_ID,Sex,Age,SRS_RRB,SRS_SCI,SRS_Total
0,0.000000,1.222222,NDARYM832PX3,1,7.048254,10.0,45.0,55.0
1,1.222222,0.555555,NDARNJ687DMC,1,6.348163,19.0,73.0,92.0
2,1.111111,1.888888,NDARED632KNG,1,17.203855,2.0,28.0,30.0
3,-0.111111,2.222222,NDARFM080VAF,1,7.623203,14.0,72.0,86.0
4,-0.111111,0.777777,NDARFW130NGG,1,8.316107,25.0,85.0,110.0
...,...,...,...,...,...,...,...,...
1234,-1.444444,-2.333333,NDARND697FLK,1,6.517796,1.0,17.0,18.0
1235,1.222222,0.444444,NDARCL008HLA,0,16.456194,21.0,82.0,103.0
1236,-1.888888,-3.000000,NDARYM334BZ5,0,16.119438,0.0,6.0,6.0
1237,-0.666666,1.333333,NDARYZ986HEW,1,7.563084,9.0,49.0,58.0


In [357]:
df_x_train_srs = pd.merge(scores, df_x_train, on='Patient_ID')
df_x_test_srs = pd.merge(scores, df_x_test, on='Patient_ID')

In [363]:
x_train_latent = df_x_train_srs.drop(['SWAN_IN_Avg' ,'SWAN_HY_Avg' , 'SRS_RRB' , 'SRS_SCI', 'SRS_Total', 'Patient_ID', 'Unnamed: 0'], axis = 1)
y_train_latent= df_x_train_srs[['SWAN_IN_Avg' ,'SWAN_HY_Avg' ,'SRS_RRB' , 'SRS_SCI', 'SRS_Total']]
x_test_latent = df_x_test_srs.drop(['SWAN_IN_Avg' ,'SWAN_HY_Avg' , 'SRS_RRB' , 'SRS_SCI', 'SRS_Total', 'Patient_ID', 'Unnamed: 0'], axis = 1)
y_test_latent =  df_x_test_srs[['SWAN_IN_Avg' ,'SWAN_HY_Avg' ,'SRS_RRB' , 'SRS_SCI', 'SRS_Total']]


In [364]:
print(x_train_latent.columns)

Index(['Sex', 'Age', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10',
       '11', '12', '13', '14'],
      dtype='object')


In [365]:
y_t_l_scale , y_l_m , y_l_s = normalize(y_train_latent)
y_t_l_scale

,SWAN_IN_Avg,SWAN_HY_Avg,SRS_RRB,SRS_SCI,SRS_Total
0,-0.672042,1.641460,0.621176,1.040625,0.970244
1,0.555704,0.973792,1.274470,0.455723,0.661655
2,-0.010948,-1.029209,0.359859,0.494716,0.476502
3,-0.483158,0.783030,0.751835,0.884651,0.877667
4,-0.766484,-0.552304,0.359859,0.182768,0.229631
...,...,...,...,...,...
711,0.839030,0.973792,-0.293435,-1.259991,-1.066441
712,0.461262,0.115363,1.535788,1.430560,1.494844
713,-2.183114,-2.841450,-1.208046,-1.532946,-1.498465
714,-1.144252,0.878411,-0.032117,0.143775,0.106196


In [366]:
checkpointer = ModelCheckpoint(filepath='Best_model_latent.h5', monitor='val_mse', verbose=1, save_best_only=True)
model = get_model(x_train_latent.shape[1] , y_t_l_scale.shape[1])
file_path = "best_model.h5"
checkpoint = ModelCheckpoint(file_path, monitor='mse', verbose=1, save_best_only=True, mode='max')

###
early = EarlyStopping(monitor="mse", mode="max", patience=10, verbose=1)
redonplat = ReduceLROnPlateau(monitor="mse", mode="max", patience=3, verbose=2)
callbacks_list = [checkpoint, early, redonplat]  # early
###
hist = model.fit(
    x_train_latent.to_numpy(), 
    y_t_l_scale.to_numpy(), 
    validation_split = 1/6,
    batch_size=275, epochs=400, verbose=2, shuffle=True, 
    callbacks=[checkpointer])

Train on 596 samples, validate on 120 samples
Epoch 1/400
 - 0s - loss: 13.1312 - mse: 13.1312 - val_loss: 11.1164 - val_mse: 11.1164

Epoch 00001: val_mse improved from inf to 11.11635, saving model to Best_model_latent.h5
Epoch 2/400
 - 0s - loss: 10.6433 - mse: 10.6433 - val_loss: 8.8346 - val_mse: 8.8346

Epoch 00002: val_mse improved from 11.11635 to 8.83460, saving model to Best_model_latent.h5
Epoch 3/400
 - 0s - loss: 8.4767 - mse: 8.4767 - val_loss: 6.9185 - val_mse: 6.9185

Epoch 00003: val_mse improved from 8.83460 to 6.91854, saving model to Best_model_latent.h5
Epoch 4/400
 - 0s - loss: 6.6605 - mse: 6.6605 - val_loss: 5.3518 - val_mse: 5.3518

Epoch 00004: val_mse improved from 6.91854 to 5.35177, saving model to Best_model_latent.h5
Epoch 5/400
 - 0s - loss: 5.1912 - mse: 5.1912 - val_loss: 4.1090 - val_mse: 4.1090

Epoch 00005: val_mse improved from 5.35177 to 4.10900, saving model to Best_model_latent.h5
Epoch 6/400
 - 0s - loss: 4.0237 - mse: 4.0237 - val_loss: 3.1538

Epoch 48/400
 - 0s - loss: 1.1146 - mse: 1.1146 - val_loss: 1.0625 - val_mse: 1.0625

Epoch 00048: val_mse improved from 1.06545 to 1.06254, saving model to Best_model_latent.h5
Epoch 49/400
 - 0s - loss: 1.1112 - mse: 1.1112 - val_loss: 1.0583 - val_mse: 1.0583

Epoch 00049: val_mse improved from 1.06254 to 1.05826, saving model to Best_model_latent.h5
Epoch 50/400
 - 0s - loss: 1.1073 - mse: 1.1073 - val_loss: 1.0553 - val_mse: 1.0553

Epoch 00050: val_mse improved from 1.05826 to 1.05532, saving model to Best_model_latent.h5
Epoch 51/400
 - 0s - loss: 1.1037 - mse: 1.1037 - val_loss: 1.0524 - val_mse: 1.0524

Epoch 00051: val_mse improved from 1.05532 to 1.05238, saving model to Best_model_latent.h5
Epoch 52/400
 - 0s - loss: 1.1003 - mse: 1.1003 - val_loss: 1.0496 - val_mse: 1.0496

Epoch 00052: val_mse improved from 1.05238 to 1.04965, saving model to Best_model_latent.h5
Epoch 53/400
 - 0s - loss: 1.0973 - mse: 1.0973 - val_loss: 1.0474 - val_mse: 1.0474

Epoch 00053: val_mse imp

 - 0s - loss: 0.9991 - mse: 0.9991 - val_loss: 0.9700 - val_mse: 0.9700

Epoch 00097: val_mse did not improve from 0.96626
Epoch 98/400
 - 0s - loss: 0.9971 - mse: 0.9971 - val_loss: 0.9744 - val_mse: 0.9744

Epoch 00098: val_mse did not improve from 0.96626
Epoch 99/400
 - 0s - loss: 0.9957 - mse: 0.9957 - val_loss: 0.9801 - val_mse: 0.9801

Epoch 00099: val_mse did not improve from 0.96626
Epoch 100/400
 - 0s - loss: 0.9957 - mse: 0.9957 - val_loss: 0.9824 - val_mse: 0.9824

Epoch 00100: val_mse did not improve from 0.96626
Epoch 101/400
 - 0s - loss: 0.9945 - mse: 0.9945 - val_loss: 0.9797 - val_mse: 0.9797

Epoch 00101: val_mse did not improve from 0.96626
Epoch 102/400
 - 0s - loss: 0.9929 - mse: 0.9929 - val_loss: 0.9749 - val_mse: 0.9749

Epoch 00102: val_mse did not improve from 0.96626
Epoch 103/400
 - 0s - loss: 0.9910 - mse: 0.9910 - val_loss: 0.9709 - val_mse: 0.9709

Epoch 00103: val_mse did not improve from 0.96626
Epoch 104/400
 - 0s - loss: 0.9893 - mse: 0.9893 - val_lo

 - 0s - loss: 0.9437 - mse: 0.9437 - val_loss: 0.9480 - val_mse: 0.9480

Epoch 00152: val_mse did not improve from 0.94088
Epoch 153/400
 - 0s - loss: 0.9431 - mse: 0.9431 - val_loss: 0.9512 - val_mse: 0.9512

Epoch 00153: val_mse did not improve from 0.94088
Epoch 154/400
 - 0s - loss: 0.9429 - mse: 0.9429 - val_loss: 0.9546 - val_mse: 0.9546

Epoch 00154: val_mse did not improve from 0.94088
Epoch 155/400
 - 0s - loss: 0.9421 - mse: 0.9421 - val_loss: 0.9557 - val_mse: 0.9557

Epoch 00155: val_mse did not improve from 0.94088
Epoch 156/400
 - 0s - loss: 0.9414 - mse: 0.9414 - val_loss: 0.9584 - val_mse: 0.9584

Epoch 00156: val_mse did not improve from 0.94088
Epoch 157/400
 - 0s - loss: 0.9423 - mse: 0.9423 - val_loss: 0.9597 - val_mse: 0.9597

Epoch 00157: val_mse did not improve from 0.94088
Epoch 158/400
 - 0s - loss: 0.9428 - mse: 0.9428 - val_loss: 0.9578 - val_mse: 0.9578

Epoch 00158: val_mse did not improve from 0.94088
Epoch 159/400
 - 0s - loss: 0.9414 - mse: 0.9414 - val_

 - 0s - loss: 0.9178 - mse: 0.9178 - val_loss: 0.9532 - val_mse: 0.9532

Epoch 00211: val_mse did not improve from 0.93469
Epoch 212/400
 - 0s - loss: 0.9185 - mse: 0.9185 - val_loss: 0.9546 - val_mse: 0.9546

Epoch 00212: val_mse did not improve from 0.93469
Epoch 213/400
 - 0s - loss: 0.9187 - mse: 0.9187 - val_loss: 0.9520 - val_mse: 0.9520

Epoch 00213: val_mse did not improve from 0.93469
Epoch 214/400
 - 0s - loss: 0.9179 - mse: 0.9179 - val_loss: 0.9492 - val_mse: 0.9492

Epoch 00214: val_mse did not improve from 0.93469
Epoch 215/400
 - 0s - loss: 0.9172 - mse: 0.9172 - val_loss: 0.9460 - val_mse: 0.9460

Epoch 00215: val_mse did not improve from 0.93469
Epoch 216/400
 - 0s - loss: 0.9161 - mse: 0.9161 - val_loss: 0.9440 - val_mse: 0.9440

Epoch 00216: val_mse did not improve from 0.93469
Epoch 217/400
 - 0s - loss: 0.9160 - mse: 0.9160 - val_loss: 0.9391 - val_mse: 0.9391

Epoch 00217: val_mse did not improve from 0.93469
Epoch 218/400
 - 0s - loss: 0.9164 - mse: 0.9164 - val_

 - 0s - loss: 0.9019 - mse: 0.9019 - val_loss: 0.9418 - val_mse: 0.9418

Epoch 00271: val_mse did not improve from 0.93469
Epoch 272/400
 - 0s - loss: 0.9019 - mse: 0.9019 - val_loss: 0.9425 - val_mse: 0.9425

Epoch 00272: val_mse did not improve from 0.93469
Epoch 273/400
 - 0s - loss: 0.9021 - mse: 0.9021 - val_loss: 0.9427 - val_mse: 0.9427

Epoch 00273: val_mse did not improve from 0.93469
Epoch 274/400
 - 0s - loss: 0.9017 - mse: 0.9017 - val_loss: 0.9457 - val_mse: 0.9457

Epoch 00274: val_mse did not improve from 0.93469
Epoch 275/400
 - 0s - loss: 0.9009 - mse: 0.9009 - val_loss: 0.9461 - val_mse: 0.9461

Epoch 00275: val_mse did not improve from 0.93469
Epoch 276/400
 - 0s - loss: 0.9009 - mse: 0.9009 - val_loss: 0.9496 - val_mse: 0.9496

Epoch 00276: val_mse did not improve from 0.93469
Epoch 277/400
 - 0s - loss: 0.9014 - mse: 0.9014 - val_loss: 0.9537 - val_mse: 0.9537

Epoch 00277: val_mse did not improve from 0.93469
Epoch 278/400
 - 0s - loss: 0.9008 - mse: 0.9008 - val_


Epoch 00330: val_mse did not improve from 0.93228
Epoch 331/400
 - 0s - loss: 0.8910 - mse: 0.8910 - val_loss: 0.9392 - val_mse: 0.9392

Epoch 00331: val_mse did not improve from 0.93228
Epoch 332/400
 - 0s - loss: 0.8905 - mse: 0.8905 - val_loss: 0.9443 - val_mse: 0.9443

Epoch 00332: val_mse did not improve from 0.93228
Epoch 333/400
 - 0s - loss: 0.8898 - mse: 0.8898 - val_loss: 0.9465 - val_mse: 0.9465

Epoch 00333: val_mse did not improve from 0.93228
Epoch 334/400
 - 0s - loss: 0.8893 - mse: 0.8893 - val_loss: 0.9448 - val_mse: 0.9448

Epoch 00334: val_mse did not improve from 0.93228
Epoch 335/400
 - 0s - loss: 0.8895 - mse: 0.8895 - val_loss: 0.9418 - val_mse: 0.9418

Epoch 00335: val_mse did not improve from 0.93228
Epoch 336/400
 - 0s - loss: 0.8911 - mse: 0.8911 - val_loss: 0.9352 - val_mse: 0.9352

Epoch 00336: val_mse did not improve from 0.93228
Epoch 337/400
 - 0s - loss: 0.8918 - mse: 0.8918 - val_loss: 0.9371 - val_mse: 0.9371

Epoch 00337: val_mse did not improve fro

 - 0s - loss: 0.8805 - mse: 0.8805 - val_loss: 0.9367 - val_mse: 0.9367

Epoch 00390: val_mse did not improve from 0.92888
Epoch 391/400
 - 0s - loss: 0.8797 - mse: 0.8797 - val_loss: 0.9330 - val_mse: 0.9330

Epoch 00391: val_mse did not improve from 0.92888
Epoch 392/400
 - 0s - loss: 0.8807 - mse: 0.8807 - val_loss: 0.9315 - val_mse: 0.9315

Epoch 00392: val_mse did not improve from 0.92888
Epoch 393/400
 - 0s - loss: 0.8834 - mse: 0.8834 - val_loss: 0.9362 - val_mse: 0.9362

Epoch 00393: val_mse did not improve from 0.92888
Epoch 394/400
 - 0s - loss: 0.8808 - mse: 0.8808 - val_loss: 0.9485 - val_mse: 0.9485

Epoch 00394: val_mse did not improve from 0.92888
Epoch 395/400
 - 0s - loss: 0.8783 - mse: 0.8783 - val_loss: 0.9601 - val_mse: 0.9601

Epoch 00395: val_mse did not improve from 0.92888
Epoch 396/400
 - 0s - loss: 0.8805 - mse: 0.8805 - val_loss: 0.9625 - val_mse: 0.9625

Epoch 00396: val_mse did not improve from 0.92888
Epoch 397/400
 - 0s - loss: 0.8800 - mse: 0.8800 - val_

In [369]:
y_pred_l = model.predict(x_test_latent.to_numpy())

In [370]:
y_test_latent_scale = normalize_p(y_test_latent , y_l_m , y_l_s)

In [371]:
for i in range(len(y_test.columns)):
    print(y_test_latent.columns[i] , ':', mean_squared_error( y_test_latent_scale.to_numpy()[:,i] , y_pred_l[:,i]) )

SWAN_IN_Avg : 1.1469614493804434
SWAN_HY_Avg : 1.0895909652144202
SRS_RRB : 0.9627861811969849
SRS_SCI : 0.8088644584625134
SRS_Total : 0.8623029868620372


## Mean Baseline For latent 

In [373]:
y_latent_mean = np.zeros(y_test_latent.shape) + np.mean(y_t_l_scale , axis=0).to_numpy()
for i in range(len(y_test.columns)):
    print(y_test_latent.columns[i] , ':', mean_squared_error( y_test_latent_scale.to_numpy()[:,i] , y_latent_mean[:,i]) )

SWAN_IN_Avg : 1.1118545009115888
SWAN_HY_Avg : 1.0224701184257916
SRS_RRB : 0.875080479818877
SRS_SCI : 0.776254368094447
SRS_Total : 0.7964931461443464
